In [ ]:
%load_ext autoreload
%autoreload 2

#  Transformation to $k_x(k_y)$ space of the RHEED image
This notebook shows how to use `Ewald` module to transform the RHEED image to the $k_x(k_y)$ space.

In [ ]:
import xrheed
from xrheed.io import load_data

import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
from pathlib import Path

## Prepare the RHEED data
Although it is not necessary it is recommended to first prepare the RHEED data by proper alignment and also test the spot position with the Ewald calculated points as shown bellow.

In [ ]:
image_dir = Path("example_data")
image_path = image_dir / "Si_111_r3Ag_112_phi_00.raw"

rheed_image = load_data(image_path, plugin="dsnp_arpes_raw")

center_x = -1.1
center_y = 0.3

rheed_image.ri.apply_image_center(center_x, center_y)
rheed_image.ri.beta = 2.85

rheed_image.ri.screen_roi_width = 60
rheed_image.ri.screen_roi_height = 60

# Use automatic levels adjustment
rheed_image.ri.plot_image(auto_levels=0.5)

plt.show()

## Use high-pass filter
It is very usefull to use high pass filter before the transformation

In [ ]:
from xrheed.preparation.filters import high_pass_filter

sigma = 3.0
threshold = 0.7

hp_rheed_image = high_pass_filter(rheed_image, sigma=sigma, threshold=threshold)

hp_rheed_image.ri.plot_image(auto_levels=1.0)
plt.show()

In [ ]:
rheed_image = hp_rheed_image

## Prepare the 2D lattice 

In [ ]:
from xrheed.kinematics.lattice import Lattice

si_111_1x1 = Lattice.from_surface_hex(a=3.84, label="1x1")
si_111_1x1.rotate(0.0)

si_111_r3 = Lattice.from_surface_hex(a=3.84 * np.sqrt(3), label="r3")
si_111_r3.rotate(30.0)

fig, ax = plt.subplots()

si_111_r3.plot_inverse(ax=ax, space_size=3, s=10, color="b")
si_111_1x1.plot_inverse(ax=ax, space_size=3, s=20, color="r")

ax.legend(loc='upper right')
plt.show()

## Create an Ewald object
Create an instance of Ewald class using generated lattice and RHEED image

In [ ]:
from xrheed.kinematics.ewald import Ewald

ew_si_111_1x1 = Ewald(si_111_1x1, rheed_image)

ew_si_111_1x1

## RHEED image with calculated spots plot
If the RHEED image is properly aligned and scaled (and the lattice matches the real crystal) there should be good match on the image, ash visible bellow.

In [ ]:
fig, ax = plt.subplots() 

ew_si_111_1x1.calculate_ewald()
ew_si_111_1x1.plot(ax=ax, 
               show_image=True,
               show_center_lines=False,
               auto_levels=1.0)
plt.show()

## Final transformation
The image could be transformed using `transform_to_kxky` function. Please note that the result image is no longer an RHEED image and `ri` accesories are not available anymore. Nonetheless, all attributes are copied form origin RHEED image.


In [ ]:
from xrheed.conversion.image import transform_image_to_kxky

trans_image = transform_image_to_kxky(rheed_image)

fig, ax = plt.subplots() 

trans_image.plot(ax=ax, vmin=5, vmax=25, add_colorbar=False)

ax.set_xlim(-5, 5)
ax.set_ylim(-3, 0.2)
ax.set_aspect(1.0)
plt.show()

## Plot with the reciprocal lattice

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6)) 

trans_image.plot(ax=ax, vmin=5, vmax=20, add_colorbar=False, cmap='gray')

si_111_r3.plot_inverse(ax=ax, facecolors='none', edgecolors='blue', s=20)
si_111_1x1.plot_inverse(ax=ax, facecolors='none', edgecolors='red', s=30)

ax.set_xlim(-5, 5)
ax.set_ylim(-4, 1)
ax.set_aspect(1.0)
ax.set_facecolor('black') 

ax.legend(loc='upper right')
plt.show()